In [ ]:
!pip install openai --quiet

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="자신의 키를 입력하세요.")

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import json

# 파일 경로 설정
base_path = '/content/drive/MyDrive/Colab Notebooks/ChatGPT'
excel_file_path = os.path.join(base_path, "samhangsi_pms_2.xlsx")
jsonl_file_name = "samhangsi_pms_2.jsonl"

# JSONL 파일로 저장할 함수 정의
def write_jsonl(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            json_record = json.dumps(item, ensure_ascii=False)
            f.write(json_record + '\n')

# Excel 파일 읽기
df = pd.read_excel(excel_file_path)

# 결과를 저장할 리스트
dialogue_pairs = []

# 각 행에 대해 대화쌍 생성
for _, row in df.iterrows():
    pair = {
        "messages": [
            {
                "role": "system",
                "content": "당신은 재치있고 웃기는 n행시를 쓰는 AI입니다. 글자수에 맞게 써주세요."
            },
            {
                "role": "user",
                "content": f"'{row['소재']}'라는 단어로 {len(row['소재'])}행시 써주세요."
            },
            {
                "role": "assistant",
                "content": row['n행시'],
                # "weight": 1
            }
        ]
    }
    dialogue_pairs.append(pair)

# JSONL 파일로 저장
write_jsonl(jsonl_file_name, dialogue_pairs)

print(f"변환이 완료되었습니다. 결과가 '{jsonl_file_name}' 파일에 저장되었습니다.")

변환이 완료되었습니다. 결과가 'samhangsi_pms_2.jsonl' 파일에 저장되었습니다.


In [ ]:
# 파인튜닝용 파일 생성하기
input_file = client.files.create(
  file=open("samhangsi_pms_2.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
# suffix 리스트 정의
suffix_list = ["pms_silhum", "pms_real"]

# 사용할 suffix 선택 (예: 리스트의 첫 번째 항목 사용)
# selected_suffix = suffix_list[0]
selected_suffix = suffix_list[1]

# fine-tuning job 생성
ft_job = client.fine_tuning.jobs.create(
    training_file=input_file.id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={
        "batch_size": 1,
        "n_epochs": 3,
        "learning_rate_multiplier": 1.8,
    },
    suffix=selected_suffix
)

In [ ]:
# 이 코드를 갱신해야 아래 것도 갱신이 됨.
ft_info = client.fine_tuning.jobs.retrieve(ft_job.id)
ft_info.id

'ftjob-IqIXAzQQYcDQChgr4YfqVL54'

In [ ]:
# 이거가 갱신
ft_info.status

'succeeded'

In [ ]:
import pickle
import os

# 기본 경로 설정 (이미 설정되었다고 가정)
base_path = '/content/drive/MyDrive/Colab Notebooks/ChatGPT'

# pickle 파일 경로
silhum_path = os.path.join(base_path, 'models_silhum.pkl')
real_path = os.path.join(base_path, 'models_real.pkl')

# 기존 데이터 불러오기 (파일이 없거나 손상되었으면 빈 리스트 생성)
def load_or_create(file_path):
    if os.path.exists(file_path):
        try:
            with open(file_path, 'rb') as f:
                return pickle.load(f)
        except (EOFError, pickle.UnpicklingError):
            print(f"Warning: 파일 '{file_path}'를 읽는 데 문제가 발생했습니다. 새로운 빈 리스트를 생성합니다.")
    return []

models_silhum = load_or_create(silhum_path)
models_real = load_or_create(real_path)

if ft_info.fine_tuned_model == None:
    model_name = None
else:
    model_name = ft_info.fine_tuned_model
# model_name = None

# 새 모델 이름 추가
if model_name == None:
    model_name = "silhum"
else:
    model_name = ft_info.fine_tuned_model

if "silhum" in model_name:
    if model_name == "silhum":
        pass
    if model_name not in models_silhum:
        models_silhum.append(model_name)
        print(f"실험모델에 '{model_name}'가 추가되었습니다.")
    else:
        print(f"'{model_name}'는 이미 실험모델 목록에 있습니다.")
else:
    if model_name not in models_real:
        models_real.append(model_name)
        print(f"사용모델에 '{model_name}'가 추가되었습니다.")
    else:
        print(f"'{model_name}'는 이미 사용모델 목록에 있습니다.")

# 업데이트된 리스트 저장
with open(silhum_path, 'wb') as f:
    pickle.dump(models_silhum, f)

with open(real_path, 'wb') as f:
    pickle.dump(models_real, f)

print(f"모델 이름이 {base_path} 경로에 저장되었습니다.")

# 저장된 리스트 출력
print("저장된 실험 모델:", models_silhum)
print("저장된 사용 모델:", models_real)

사용모델에 'ft:gpt-4o-mini-2024-07-18:personal:pms-real:AGf4AQYG'가 추가되었습니다.
모델 이름이 /content/drive/MyDrive/Colab Notebooks/ChatGPT 경로에 저장되었습니다.
저장된 실험 모델: ['ft:gpt-4o-mini-2024-07-18:personal:pms-silhum:AGVVwYGD', 'ft:gpt-4o-mini-2024-07-18:personal:pms-silhum:AGVw0T5G', 'ft:gpt-4o-mini-2024-07-18:personal:pms-silhum:AGWA4rJA', 'ft:gpt-4o-mini-2024-07-18:personal:pms-silhum:AGWZqOFY', 'silhum', 'ft:gpt-4o-mini-2024-07-18:personal:pms-silhum:AGWus2va', 'ft:gpt-4o-mini-2024-07-18:personal:pms-silhum:AGX5BmV3']
저장된 사용 모델: ['ft:gpt-4o-mini-2024-07-18:personal:pms-real:AGf4AQYG']


In [ ]:
import csv
from datetime import datetime

stuff = input("소재를 입력해주세요 : ")

stream = client.chat.completions.create(
      # model=models_silhum[-1],
      model=models_real[-1],
      temperature=0.9,
      top_p=0.97,
      stream=True,
      messages=[
        {"role": "system", "content": "당신은 재치있고 웃기는 n행시를 쓰는 AI입니다. 글자수에 맞게 써주세요."},
        {"role": "user", "content": f"'{stuff}'라는 단어로 {len(stuff)}행시 써주세요."}
      ]
    )

print("----------------------------------")

full_content = ""
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        content = chunk.choices[0].delta.content
        print(content, end="")
        full_content += content

# CSV 파일 이름 설정
csv_filename = "n행시_결과.csv"

# 현재 시간을 가져옵니다
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# CSV 파일에 모든 내용을 저장합니다
with open(csv_filename, 'a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # 파일이 비어있다면 헤더를 추가합니다
    file.seek(0, 2)
    if file.tell() == 0:
        writer.writerow(["시간", "소재", "내용"])

    # 소재와 n행시를 같은 행에 저장합니다
    writer.writerow([current_time, stuff, full_content])

print(f"\n\nCSV 파일에 결과가 추가되었습니다: {csv_filename}")

소재를 입력해주세요 : 김정은
----------------------------------
김치보다
정말
은근히 좋다

CSV 파일에 결과가 추가되었습니다: n행시_결과.csv


In [ ]:
import csv
from datetime import datetime

# CSV 파일 이름 설정
csv_filename = "n행시_결과.csv"

# 반복 횟수 설정
repeat_count = 5  # 원하는 반복 횟수로 변경 가능

for _ in range(repeat_count):
    stuff = client.chat.completions.create(
          model="gpt-4o-mini-2024-07-18",
          temperature=0.7,
          messages=[
            {"role": "system", "content": "당신은 랜덤하게 단어를 고르는 AI입니다. '사과' 같이 딱 단어만 말하세요."},
            {"role": "user", "content": "n행시 하기 좋은 단어 하나를 골라주세요. 자연과 관련된 단어 하시면 안됩니다."},
            # {"role": "assistant", "content": "사랑"},
            # {"role": "user", "content": "좋아요. 다른 좋은 단어 골라주세요."},
            # {"role": "assistant", "content": "비행기"},
            # {"role": "user", "content": "좋아요. 글자수는 자유롭게 다른 좋은 단어 골라주세요."},
          ]
        )

    stuff = stuff.choices[0].message.content
    print(f"소재는 {stuff} 입니다.")

    stream = client.chat.completions.create(
          model=models_silhum[-1],
          temperature=0.8,
          top_p=0.97,
          stream=True,
          messages=[
            {"role": "system", "content": "당신은 재치있고 웃기는 n행시를 쓰는 AI입니다."},
            {"role": "user", "content": f"'{stuff}'라는 단어로 n행시 써주세요."}
          ]
        )

    print("----------------------------------")

    full_content = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="")
            full_content += content

    # 현재 시간을 가져옵니다
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # CSV 파일에 모든 내용을 저장합니다
    with open(csv_filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # 파일이 비어있다면 헤더를 추가합니다
        file.seek(0, 2)
        if file.tell() == 0:
            writer.writerow(["시간", "소재", "내용"])

        # 소재와 n행시를 같은 행에 저장합니다
        writer.writerow([current_time, stuff, full_content])

    print(f"\n\nCSV 파일에 결과가 추가되었습니다: {csv_filename}")

print("\n모든 n행시 생성이 완료되었습니다.")

소재는 사랑 입니다.
----------------------------------
사랑은
아프다
랑...

CSV 파일에 결과가 추가되었습니다: n행시_결과.csv
소재는 도시 입니다.
----------------------------------
도대체
시골은 왜
좋은 걸까

CSV 파일에 결과가 추가되었습니다: n행시_결과.csv
소재는 사랑 입니다.
----------------------------------
사랑하자
라면도 끓여주고
면도 해주고
주말마다
말도 안 되게
해주고 싶다

CSV 파일에 결과가 추가되었습니다: n행시_결과.csv
소재는 사람 입니다.
----------------------------------
사랑하면
람쥐도 단다

CSV 파일에 결과가 추가되었습니다: n행시_결과.csv
소재는 사랑 입니다.
----------------------------------
사랑해
라면도
면도기처럼
기억할게
냥냥펀치

CSV 파일에 결과가 추가되었습니다: n행시_결과.csv

모든 n행시 생성이 완료되었습니다.


In [ ]:
import csv
from datetime import datetime

stuff = client.chat.completions.create(
      model="gpt-4o-mini-2024-07-18",
      temperature=0.6,
      top_p=0.97,
      messages=[
        {"role": "system", "content": "당신은 단어를 고르는 AI입니다. 좋은 단어를 아무거나 골라주세요."},
        {"role": "user", "content": "세 글자 단어 하나만 골라주세요."},
        {"role": "assistant", "content": "자동차"},
        {"role": "user", "content": "좋아요. 그런 식으로 세 글자 단어 하나 골라주세요."},
      ]
    )

stuff = stuff.choices[0].message.content
print(f"소재는 {stuff} 입니다.")

stream = client.chat.completions.create(
      model=models_silhum[-1],
      temperature=0.5,
      top_p=0.97,
      stream=True,
      messages=[
        {"role": "system", "content": "당신은 재치있고 웃기는 n행시를 쓰는 AI입니다."},
        {"role": "user", "content": f"'{stuff}'라는 단어로 n행시 써주세요."}
      ]
    )

print("----------------------------------")

full_content = ""
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        content = chunk.choices[0].delta.content
        print(content, end="")
        full_content += content

# CSV 파일 이름 설정
csv_filename = "n행시_결과.csv"

# 현재 시간을 가져옵니다
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# CSV 파일에 모든 내용을 저장합니다
with open(csv_filename, 'a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # 파일이 비어있다면 헤더를 추가합니다
    file.seek(0, 2)
    if file.tell() == 0:
        writer.writerow(["시간", "소재", "내용"])

    # 소재와 n행시를 같은 행에 저장합니다
    writer.writerow([current_time, stuff, full_content])

print(f"\n\nCSV 파일에 결과가 추가되었습니다: {csv_filename}")

소재는 하늘이 입니다.
----------------------------------
하늘이
너무 예쁘다

CSV 파일에 결과가 추가되었습니다: n행시_결과.csv
